In [17]:
import mahotas as mh
from sklearn import cross_validation
from sklearn.linear_model import ElasticNet
#from sklearn.pipeline import Pipeline
#from sklearn.preprocessing import StandardScaler
#from sklearn.grid_search import GridSearchCV
import numpy as np
import os
import pandas
from sklearn.linear_model import ElasticNetCV
from sklearn.cluster import KMeans
from mahotas.features import surf

path = '~/'
# basedir = '/home/antigrav/work/skinhack/images'
basedir = '/volume/data/age_test_1600/age_test/'
csv_pth = "/volume/data/age_test_strip.csv"
# csv_pth = "/home/antigrav/work/skinhack/age.csv"


def texture(im):
    '''Compute features for an image

    Parameters
    ----------
    im : ndarray

    Returns
    -------
    fs : ndarray
        1-D array of features
    '''
    im = im.astype(np.uint8)
    return mh.features.haralick(im).ravel()

def chist(im):
    '''Compute color histogram of input image

    Parameters
    ----------
    im : ndarray
        should be an RGB image

    Returns
    -------
    c : ndarray
        1-D array of histogram values
    '''

    # Downsample pixel values:
    im = im // 64

    # Separate RGB channels:
    r, g, b = im.transpose((2, 0, 1))

    pixels = 1 * r + 4 * g + 16 * b
    hist = np.bincount(pixels.ravel(), minlength=64)
    hist = hist.astype(float)
    return np.log1p(hist)

#help(ElasticNetCV)
enet=ElasticNetCV(n_alphas=10) #чем больше n_alhas тем лучше
#alpha_range = 2**np.arange(-5, 5)
#clf=GridSearchCV(ElasticNet(l1_ratio=0.5), param_grid={'alpha': alpha_range})

#grid = GridSearchCV(ElasticNet(l1_ratio=0.5), param_grid={'alpha': alpha_range})
#clf = Pipeline([('preproc', StandardScaler()),('classifier', grid)])

In [20]:
data = pandas.read_csv(csv_pth, sep=",")
#data = data[data.age != 0]
data = data.drop_duplicates(subset = "image")


#images = [basedir + '/' + i for i in os.listdir(basedir)]
#numberofimagefortest=30

ifeatures = []
import sys

#n_min=int(0)
#n_max=int(sys.argv[2])
n_min=0
n_max=10000
#images = images[:numberofimagefortest]
data=data.iloc[n_min:n_max]
#data.photo = data.photo[:n_img] #[basedir + '/' + i for i in data.photo]
#data.age = data.age[:n_img]
#Y = data.age

#Y.to_csv(path + '/Y_test.csv')
#data.gender.to_csv(path + '/gender_test.csv')
data.area.to_csv(path + '/area_test.csv')

#print (data.photo) #[:n_img]
#print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
Y=np.array(Y,dtype=float)
#Слабое место, оптимизировать необходимо
#pritn(data.photo)
#data.age.to_csv(path + "bar2.csv")
#exit(0)

In [21]:
import pandas as pd

out = pd.DataFrame()
#out['Y'] = Y
#out['gender'] = data.gender
out['area'] = data.area

In [22]:
out.to_csv(path + 'out_test.csv')

In [23]:
alldescriptors = []
#Супермедленная хрень, оптимизировать
sp=64 #Сколько нужно отступать в пикселях, чем больше, тем быстрее, но менее точнее.
bad_time = []
for num, fname in enumerate(data.image):
    print num
    try:
        print fname
        imc = mh.imread(fname)
        im = mh.colors.rgb2grey(imc)
        alldescriptors.append(surf.dense(im, spacing=200))
    except ValueError:
        print(fname + " bad name")
        bad_time.append(num)
    # use dense sampling, you can try the following line:
    # alldescriptors.append(surf.dense(im, spacing=16))
    # alldescriptors.append(surf.surf(im, descriptor_only=True))

print('Descriptor computation complete.')

0
/volume/data/age_test_1600/age_test/031235005020003401.tif-0.jpg
1
/volume/data/age_test_1600/age_test/031235005010003301.tif-0.jpg
2
/volume/data/age_test_1600/age_test/031235006010003401.tif-0.jpg
3
/volume/data/age_test_1600/age_test/031235006020003301.tif-0.jpg
4
/volume/data/age_test_1600/age_test/031235007010003301.tif-0.jpg
5
/volume/data/age_test_1600/age_test/031235007020003401.tif-0.jpg
6
/volume/data/age_test_1600/age_test/031235008010003401.tif-0.jpg
7
/volume/data/age_test_1600/age_test/031235008020003301.tif-0.jpg
8
/volume/data/age_test_1600/age_test/031235009010003301.tif-0.jpg
9
/volume/data/age_test_1600/age_test/031235009020003401.tif-0.jpg
10
/volume/data/age_test_1600/age_test/031236001010003301.tif-0.jpg
11
/volume/data/age_test_1600/age_test/031236001020003401.tif-0.jpg
12
/volume/data/age_test_1600/age_test/031236010010003401.tif-0.jpg
13
/volume/data/age_test_1600/age_test/031236010020003301.tif-0.jpg
14
/volume/data/age_test_1600/age_test/031236011010003301.

In [26]:
np.shape(alldescriptors)

(1469, 45, 64)

In [24]:
pddescs = pd.DataFrame(alldescriptors)

ValueError: Must pass 2-d input

In [ ]:
pddescs.to_csv(path + 'test_descriptors.csv', index=False)

In [27]:
concatenated = np.concatenate(alldescriptors)

In [28]:
pd.DataFrame(concatenated).to_csv(path + 'test_concatenated.csv')

In [31]:
import tqdm

ifeatures = []

for fname in tqdm.tqdm(data.image):
    try:
        imc = mh.imread(fname)
        im = mh.colors.rgb2grey(imc)
        ifeatures.append(np.concatenate([texture(im), chist(imc)]))
    except ValueError:
        print(fname)
print('Alg stop')

 62%|██████▏   | 912/1470 [08:32<04:26,  2.09it/s]

/volume/data/age_test_1600/age_test/034503013020001601.tif-0.jpg


100%|██████████| 1470/1470 [14:01<00:00,  1.48it/s]

Alg stop


In [33]:
k =  16
km = KMeans(k)

km.fit(concatenated)
sfeatures = []
for d in alldescriptors:
    c = km.predict(d)
    sfeatures.append(np.bincount(c, minlength=k))
sfeatures = np.array(sfeatures, dtype=float)

allfeatures = np.hstack([sfeatures, ifeatures])

In [38]:
pd.DataFrame(allfeatures).to_csv(path + 'test_features.csv', index=False)

In [36]:
pd.DataFrame(allfeatures)

,0,1,2,3,4,5,6,7,8,9,...,122,123,124,125,126,127,128,129,130,131
0,5.0,1.0,2.0,2.0,3.0,5.0,1.0,0.0,3.0,7.0,...,0.0,0.0,0.0,0.0,1.609438,0.693147,0.0,0.0,1.386294,6.135565
1,3.0,0.0,3.0,1.0,5.0,4.0,2.0,1.0,2.0,5.0,...,0.0,0.0,0.0,0.0,1.098612,1.098612,0.0,0.0,1.098612,6.469250
2,6.0,0.0,2.0,4.0,5.0,3.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.693147,1.098612,0.0,0.0,1.386294,6.862758
3,4.0,1.0,2.0,3.0,4.0,3.0,3.0,0.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1.098612,6.820016
4,3.0,3.0,1.0,0.0,7.0,3.0,2.0,2.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.000000,1.609438,0.0,0.0,0.000000,6.985642
5,3.0,0.0,3.0,3.0,3.0,1.0,0.0,3.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.000000,0.693147,0.0,0.0,0.000000,6.257668
6,3.0,0.0,2.0,4.0,7.0,3.0,3.0,2.0,3.0,2.0,...,0.0,0.0,0.0,0.0,2.484907,0.000000,0.0,0.0,3.044522,6.946976
7,4.0,3.0,3.0,5.0,5.0,2.0,9.0,3.0,1.0,1.0,...,0.0,0.0,0.0,0.0,2.079442,0.693147,0.0,0.0,3.178054,7.154615
8,2.0,2.0,2.0,2.0,4.0,2.0,3.0,1.0,6.0,3.0,...,0.0,0.0,0.0,0.0,1.098612,0.693147,0.0,0.0,2.639057,8.406485
9,4.0,2.0,2.0,6.0,1.0,3.0,2.0,0.0,4.0,3.0,...,0.0,0.0,0.0,0.0,1.098612,1.609438,0.0,0.0,2.484907,8.239329


In [39]:
test

NameError: name 'test' is not defined